# Coherence Alignment Analysis for 3I/ATLAS
This notebook reproduces the key analyses and figures described in the manuscript *"Statistical Evidence for Coherence-Weighted Trajectories"*.

**Workflow overview:**
1. Import required libraries
2. Load orbital and reference-plane data
3. Perform coordinate transformations
4. Run isotropic Monte Carlo simulations
5. Compute alignment probabilities
6. Plot results


In [ ]:
# Step 1 — Imports
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from numpy.random import default_rng
from astropy import units as u
from astropy.coordinates import CartesianRepresentation, spherical_to_cartesian

rng = default_rng(seed=42)
print('Environment initialized.')

In [ ]:
# Step 2 — Load data
orbital_df = pd.read_csv('../data/orbital_elements/3I_ATLAS_MPC.csv')
with open('../data/reference_planes/JUPITER_LAPLACE.json') as f:
    ref_planes = json.load(f)

orbital_df, ref_planes

In [ ]:
# Step 3 — Coordinate transformations (simplified placeholder)
def rotation_matrix(i, Omega, omega):
    Rz1 = np.array([[np.cos(Omega), -np.sin(Omega), 0], [np.sin(Omega), np.cos(Omega), 0], [0, 0, 1]])
    Rx = np.array([[1, 0, 0], [0, np.cos(i), -np.sin(i)], [0, np.sin(i), np.cos(i)]])
    Rz2 = np.array([[np.cos(omega), -np.sin(omega), 0], [np.sin(omega), np.cos(omega), 0], [0, 0, 1]])
    return Rz1 @ Rx @ Rz2

elements = orbital_df.iloc[0]
rot = rotation_matrix(np.radians(elements.i), np.radians(elements.Omega), np.radians(elements.omega))
rot

In [ ]:
# Step 4 — Monte Carlo isotropic sampling
N = 10**6
theta = np.arccos(1 - 2 * rng.random(N))  # isotropic in cos(theta)
phi = 2 * np.pi * rng.random(N)

x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

vectors = np.vstack((x, y, z)).T
vectors.shape

In [ ]:
# Step 5 — Compute probability of alignment within threshold
def angle_to_plane(vectors, plane_normal):
    dot_products = np.abs(np.dot(vectors, plane_normal)) / np.linalg.norm(plane_normal)
    angles = np.degrees(90 - np.degrees(np.arccos(dot_products)))
    return angles

plane_normal = np.array([0, 0, 1])  # ecliptic approximation
angles = angle_to_plane(vectors, plane_normal)

theta0 = 2.5  # degrees
p_within = np.mean(np.abs(angles) <= theta0)
print(f'P(|θ| ≤ {theta0}°) = {p_within:.4f}')

In [ ]:
# Step 6 — Plot results
plt.hist(np.abs(angles), bins=200, density=True, color='royalblue', alpha=0.7)
plt.axvline(theta0, color='crimson', linestyle='--', label=f'Observed (2.5°)')
plt.xlabel('|θ| (deg)')
plt.ylabel('Probability Density')
plt.title('Isotropic Plane-Angle Distribution')
plt.legend()
plt.show()